In [31]:
import pandas as pd
import json
from datetime import timedelta, datetime, date
import os


## Get csv file from jwa_rawdata

In [32]:
def display_path(path):
    return path.replace("\\", "/")


def concat_files(folder, func):
    result = []
    for year in os.listdir(folder):
        for month in os.listdir(os.path.join(folder, year)):
            for day in os.listdir(os.path.join(folder, year, month)):
                for filename in os.listdir(os.path.join(folder, year, month, day)):
                    df = func(
                        display_path(os.path.join(folder, year, month, day, filename))
                    )
                    result.append(df)
    return pd.concat(result)


def get_weather(filename):
    file = json.load(open(filename, "rb"))
    values = file["results"][0]["values"]
    forecast_time = [value["forecast_time"] for value in values]
    weather = [value["weather"]["value"] for value in values]
    device_id = filename.split(".")[0].split("#")[-1]
    return pd.DataFrame(
        {
            "device_id": device_id,
            "forecast_time": forecast_time,
            "weather": weather,
        }
    )


In [33]:
def get_humidity(filename):
    file = json.load(open(filename, "rb"))
    prodIdInfo = file["product"][0]
    forecasts = prodIdInfo["point"][0]["forecast"]
    forecast_time = [forecast["time"] for forecast in forecasts]
    Precip = [forecast["Precip"][0] for forecast in forecasts]
    humidity = [forecast["humidity"][0] for forecast in forecasts]
    return pd.DataFrame(
        {
            "forecast_time": forecast_time,
            "Precip": Precip,
            "humidity": humidity,
        }
    )


In [34]:
def get_solar_pv_output(filename):
    file = json.load(open(filename, "rb"))
    values = file["results"][0]["values"]
    forecast_time = [value["forecast_time"] for value in values]
    pv_output = [value["pv_output"]["value"] for value in values]
    device_id = filename.split(".")[0].split("#")[-1]
    return pd.DataFrame(
        {
            "device_id": device_id,
            "forecast_time": forecast_time,
            "pv_output": pv_output,
        }
    )


In [35]:
def get_solar_radiation(filename):
    file = json.load(open(filename, "rb"))
    values = file["results"][0]["values"]
    forecast_time = [value["forecast_time"] for value in values]
    solar_radiation = [value["solar_radiation"]["value"] for value in values]
    device_id = filename.split(".")[0].split("#")[-1]
    return pd.DataFrame(
        {
            "device_id": device_id,
            "forecast_time": forecast_time,
            "solar_radiation": solar_radiation,
        }
    )


In [36]:
def get_temperature(filename):
    file = json.load(open(filename, "rb"))
    values = file["results"][0]["values"]
    forecast_time = [value["forecast_time"] for value in values]
    temperature = [value["temperature"]["value"] for value in values]
    device_id = filename.split(".")[0].split("#")[-1]
    return pd.DataFrame(
        {
            "device_id": device_id,
            "forecast_time": forecast_time,
            "temperature": temperature,
        }
    )


## Convert time from GMT0 to GMT9

In [37]:
def add_hours(df, hours=9):
    df["forecast_time"] = [
        datetime.fromisoformat(i[:-1]) + timedelta(hours=hours)
        for i in df["forecast_time"]
    ]
    return df


In [38]:
# base_dir = 'data/timezone9/origin_data'
# for filename in os.listdir(base_dir):
#     df = pd.read_csv(os.path.join(base_dir, filename))
#     df = add_hours(df)
#     df.to_csv('data/timezone9/' + filename, index=False)


In [39]:
target = pd.read_csv("data/timezone9/target.csv")
target.drop_duplicates(inplace=True)
target = target.groupby(["device_id", "timestamp", "lat_long"], as_index=False).mean()

target["timestamp"] = pd.to_datetime(target["timestamp"])
timestamp = pd.to_datetime(target["timestamp"])
target["timestamp_round1h"] = timestamp.dt.floor("H")
target["timestamp_round30m"] = timestamp.dt.floor("30min")
target_cols = target.columns.to_list()
target


,device_id,timestamp,lat_long,value,timestamp_round1h,timestamp_round30m
0,customer_1,2021-10-04 20:09:38,34.7037561598_137.3584203017,2858.0,2021-10-04 20:00:00,2021-10-04 20:00:00
1,customer_1,2021-10-04 22:39:38,34.7037561598_137.3584203017,2958.0,2021-10-04 22:00:00,2021-10-04 22:30:00
2,customer_1,2021-10-04 22:49:38,34.7037561598_137.3584203017,2683.0,2021-10-04 22:00:00,2021-10-04 22:30:00
3,customer_1,2021-10-04 22:59:38,34.7037561598_137.3584203017,2374.0,2021-10-04 22:00:00,2021-10-04 22:30:00
4,customer_1,2021-10-05 03:39:38,34.7037561598_137.3584203017,1808.0,2021-10-05 03:00:00,2021-10-05 03:30:00
...,...,...,...,...,...,...
87835,poweru,2022-04-17 23:18:50,34.7037561598_137.3584203017,194.3,2022-04-17 23:00:00,2022-04-17 23:00:00
87836,poweru,2022-04-17 23:28:50,34.7037561598_137.3584203017,181.8,2022-04-17 23:00:00,2022-04-17 23:00:00
87837,poweru,2022-04-17 23:38:50,34.7037561598_137.3584203017,209.7,2022-04-17 23:00:00,2022-04-17 23:30:00
87838,poweru,2022-04-17 23:48:50,34.7037561598_137.3584203017,209.7,2022-04-17 23:00:00,2022-04-17 23:30:00


## Merge solar_pv_output with solar_radiation

In [40]:
solar_pv_output = pd.read_csv("data/timezone9/solar_pv_output.csv")
solar_pv_output["forecast_time"] = pd.to_datetime(solar_pv_output["forecast_time"])
solar_pv_output


,device_id,forecast_time,pv_output
0,customer_1,2022-01-28 09:30:00,46.206001
1,customer_1,2022-01-28 10:00:00,51.565622
2,customer_1,2022-01-28 10:30:00,54.478857
3,customer_1,2022-01-28 11:00:00,60.751781
4,customer_1,2022-01-28 11:30:00,67.629492
...,...,...,...
305443,poweru,2022-04-21 13:00:00,32.058677
305444,poweru,2022-04-21 13:30:00,29.966487
305445,poweru,2022-04-21 14:00:00,30.669357
305446,poweru,2022-04-21 14:30:00,27.973101


In [41]:
solar_radiation = pd.read_csv("data/timezone9/solar_radiation.csv")
solar_radiation["forecast_time"] = pd.to_datetime(solar_radiation["forecast_time"])
solar_radiation


,device_id,forecast_time,solar_radiation
0,customer_1,2022-01-28 09:30:00,353.0
1,customer_1,2022-01-28 10:00:00,409.0
2,customer_1,2022-01-28 10:30:00,447.0
3,customer_1,2022-01-28 11:00:00,500.0
4,customer_1,2022-01-28 11:30:00,553.0
...,...,...,...
305443,poweru,2022-04-21 13:00:00,373.0
305444,poweru,2022-04-21 13:30:00,349.0
305445,poweru,2022-04-21 14:00:00,357.0
305446,poweru,2022-04-21 14:30:00,326.0


In [42]:
spo_sr = solar_pv_output.merge(
    solar_radiation, how="left", on=["device_id", "forecast_time"]
)
spo_sr.drop_duplicates(inplace=True)
spo_sr = spo_sr.groupby(["device_id", "forecast_time"], as_index=False).mean()
spo_sr.columns = ["device_id", "timestamp_round30m", "pv_output", "solar_radiation"]
spo_sr


,device_id,timestamp_round30m,pv_output,solar_radiation
0,PowerU,2022-03-02 09:30:00,56.464004,506.0
1,PowerU,2022-03-02 10:00:00,63.726411,580.0
2,PowerU,2022-03-02 10:30:00,69.006336,637.0
3,PowerU,2022-03-02 11:00:00,72.996559,681.0
4,PowerU,2022-03-02 11:30:00,76.150162,715.0
...,...,...,...,...
27049,poweru,2022-04-21 13:00:00,32.058677,373.0
27050,poweru,2022-04-21 13:30:00,29.966487,349.0
27051,poweru,2022-04-21 14:00:00,30.669357,357.0
27052,poweru,2022-04-21 14:30:00,27.973101,326.0


## Merge target with solar_pv_output and solar_radiation

In [43]:
target_spo_sr = target.merge(spo_sr, how="left", on=["device_id", "timestamp_round30m"])
target_spo_sr.drop_duplicates(inplace=True)
target_spo_sr


,device_id,timestamp,lat_long,value,timestamp_round1h,timestamp_round30m,pv_output,solar_radiation
0,customer_1,2021-10-04 20:09:38,34.7037561598_137.3584203017,2858.0,2021-10-04 20:00:00,2021-10-04 20:00:00,NaN,NaN
1,customer_1,2021-10-04 22:39:38,34.7037561598_137.3584203017,2958.0,2021-10-04 22:00:00,2021-10-04 22:30:00,NaN,NaN
2,customer_1,2021-10-04 22:49:38,34.7037561598_137.3584203017,2683.0,2021-10-04 22:00:00,2021-10-04 22:30:00,NaN,NaN
3,customer_1,2021-10-04 22:59:38,34.7037561598_137.3584203017,2374.0,2021-10-04 22:00:00,2021-10-04 22:30:00,NaN,NaN
4,customer_1,2021-10-05 03:39:38,34.7037561598_137.3584203017,1808.0,2021-10-05 03:00:00,2021-10-05 03:30:00,NaN,NaN
...,...,...,...,...,...,...,...,...
87835,poweru,2022-04-17 23:18:50,34.7037561598_137.3584203017,194.3,2022-04-17 23:00:00,2022-04-17 23:00:00,0.0,0.0
87836,poweru,2022-04-17 23:28:50,34.7037561598_137.3584203017,181.8,2022-04-17 23:00:00,2022-04-17 23:00:00,0.0,0.0
87837,poweru,2022-04-17 23:38:50,34.7037561598_137.3584203017,209.7,2022-04-17 23:00:00,2022-04-17 23:30:00,0.0,0.0
87838,poweru,2022-04-17 23:48:50,34.7037561598_137.3584203017,209.7,2022-04-17 23:00:00,2022-04-17 23:30:00,0.0,0.0


## Merge temperature and weather

In [44]:
temperature = pd.read_csv("data/timezone9/temperature.csv")
temperature["forecast_time"] = pd.to_datetime(temperature["forecast_time"])
temperature


,device_id,forecast_time,temperature
0,customer_1,2022-01-28 10:00:00,6.5
1,customer_1,2022-01-28 11:00:00,7.6
2,customer_1,2022-01-28 12:00:00,8.2
3,customer_1,2022-01-28 13:00:00,8.9
4,customer_1,2022-01-28 14:00:00,9.6
...,...,...,...
152719,poweru,2022-04-21 11:00:00,19.4
152720,poweru,2022-04-21 12:00:00,19.8
152721,poweru,2022-04-21 13:00:00,19.9
152722,poweru,2022-04-21 14:00:00,19.8


In [45]:
weather = pd.read_csv("data/timezone9/weather.csv")
weather["forecast_time"] = pd.to_datetime(weather["forecast_time"])
weather


,device_id,forecast_time,weather
0,customer_1,2022-01-28 10:00:00,clear
1,customer_1,2022-01-28 11:00:00,clear
2,customer_1,2022-01-28 12:00:00,clear
3,customer_1,2022-01-28 13:00:00,clear
4,customer_1,2022-01-28 14:00:00,clear
...,...,...,...
152719,poweru,2022-04-21 11:00:00,clear
152720,poweru,2022-04-21 12:00:00,clear
152721,poweru,2022-04-21 13:00:00,clear
152722,poweru,2022-04-21 14:00:00,clear


In [46]:
temp_wea = temperature.merge(weather, how="left", on=["device_id", "forecast_time"])
temp_wea.drop_duplicates(inplace=True)
temp_wea = temp_wea.groupby(["device_id", "forecast_time"], as_index=False).mean()
temp_wea.columns = ["device_id", "timestamp_round1h", "temperature"]
temp_wea


,device_id,timestamp_round1h,temperature
0,PowerU,2022-03-02 10:00:00,10.7
1,PowerU,2022-03-02 11:00:00,12.2
2,PowerU,2022-03-02 12:00:00,13.1
3,PowerU,2022-03-02 13:00:00,13.8
4,PowerU,2022-03-02 14:00:00,13.8
...,...,...,...
13522,poweru,2022-04-21 11:00:00,19.4
13523,poweru,2022-04-21 12:00:00,19.8
13524,poweru,2022-04-21 13:00:00,19.9
13525,poweru,2022-04-21 14:00:00,19.8


## Merge target_spo_sr with temperature and weather

In [47]:
target_spo_sr_temp_wea = target_spo_sr.merge(
    temp_wea, how="left", on=["device_id", "timestamp_round1h"]
)
target_spo_sr_temp_wea.drop_duplicates(inplace=True)
target_spo_sr_temp_wea


,device_id,timestamp,lat_long,value,timestamp_round1h,timestamp_round30m,pv_output,solar_radiation,temperature
0,customer_1,2021-10-04 20:09:38,34.7037561598_137.3584203017,2858.0,2021-10-04 20:00:00,2021-10-04 20:00:00,NaN,NaN,NaN
1,customer_1,2021-10-04 22:39:38,34.7037561598_137.3584203017,2958.0,2021-10-04 22:00:00,2021-10-04 22:30:00,NaN,NaN,NaN
2,customer_1,2021-10-04 22:49:38,34.7037561598_137.3584203017,2683.0,2021-10-04 22:00:00,2021-10-04 22:30:00,NaN,NaN,NaN
3,customer_1,2021-10-04 22:59:38,34.7037561598_137.3584203017,2374.0,2021-10-04 22:00:00,2021-10-04 22:30:00,NaN,NaN,NaN
4,customer_1,2021-10-05 03:39:38,34.7037561598_137.3584203017,1808.0,2021-10-05 03:00:00,2021-10-05 03:30:00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
87835,poweru,2022-04-17 23:18:50,34.7037561598_137.3584203017,194.3,2022-04-17 23:00:00,2022-04-17 23:00:00,0.0,0.0,13.56
87836,poweru,2022-04-17 23:28:50,34.7037561598_137.3584203017,181.8,2022-04-17 23:00:00,2022-04-17 23:00:00,0.0,0.0,13.56
87837,poweru,2022-04-17 23:38:50,34.7037561598_137.3584203017,209.7,2022-04-17 23:00:00,2022-04-17 23:30:00,0.0,0.0,13.56
87838,poweru,2022-04-17 23:48:50,34.7037561598_137.3584203017,209.7,2022-04-17 23:00:00,2022-04-17 23:30:00,0.0,0.0,13.56


## Merge with humidity

In [48]:
humidity = pd.read_csv("data/timezone9/humidity.csv")
humidity["forecast_time"] = pd.to_datetime(humidity["forecast_time"]).dt.tz_convert(
    None
) + timedelta(hours=9)
humidity = humidity.groupby(["forecast_time"], as_index=False).mean()
humidity.columns = ["timestamp_round1h", "Precip", "humidity"]
humidity


,timestamp_round1h,Precip,humidity
0,2022-01-28 10:00:00,0.0,54.0
1,2022-01-28 11:00:00,0.0,56.0
2,2022-01-28 12:00:00,0.0,54.0
3,2022-01-28 13:00:00,0.0,54.0
4,2022-01-28 14:00:00,0.0,52.0
...,...,...,...
1993,2022-04-21 11:00:00,0.0,62.0
1994,2022-04-21 12:00:00,0.0,66.0
1995,2022-04-21 13:00:00,0.0,68.0
1996,2022-04-21 14:00:00,0.0,70.0


In [49]:
full_features = target_spo_sr_temp_wea.merge(
    humidity, how="left", on=["timestamp_round1h"]
)
full_features.drop_duplicates(inplace=True)
full_features


,device_id,timestamp,lat_long,value,timestamp_round1h,timestamp_round30m,pv_output,solar_radiation,temperature,Precip,humidity
0,customer_1,2021-10-04 20:09:38,34.7037561598_137.3584203017,2858.0,2021-10-04 20:00:00,2021-10-04 20:00:00,NaN,NaN,NaN,NaN,NaN
1,customer_1,2021-10-04 22:39:38,34.7037561598_137.3584203017,2958.0,2021-10-04 22:00:00,2021-10-04 22:30:00,NaN,NaN,NaN,NaN,NaN
2,customer_1,2021-10-04 22:49:38,34.7037561598_137.3584203017,2683.0,2021-10-04 22:00:00,2021-10-04 22:30:00,NaN,NaN,NaN,NaN,NaN
3,customer_1,2021-10-04 22:59:38,34.7037561598_137.3584203017,2374.0,2021-10-04 22:00:00,2021-10-04 22:30:00,NaN,NaN,NaN,NaN,NaN
4,customer_1,2021-10-05 03:39:38,34.7037561598_137.3584203017,1808.0,2021-10-05 03:00:00,2021-10-05 03:30:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
87835,poweru,2022-04-17 23:18:50,34.7037561598_137.3584203017,194.3,2022-04-17 23:00:00,2022-04-17 23:00:00,0.0,0.0,13.56,0.038462,84.923077
87836,poweru,2022-04-17 23:28:50,34.7037561598_137.3584203017,181.8,2022-04-17 23:00:00,2022-04-17 23:00:00,0.0,0.0,13.56,0.038462,84.923077
87837,poweru,2022-04-17 23:38:50,34.7037561598_137.3584203017,209.7,2022-04-17 23:00:00,2022-04-17 23:30:00,0.0,0.0,13.56,0.038462,84.923077
87838,poweru,2022-04-17 23:48:50,34.7037561598_137.3584203017,209.7,2022-04-17 23:00:00,2022-04-17 23:30:00,0.0,0.0,13.56,0.038462,84.923077


### Add feature day_of_week,working_day,week_of_month,holiday

In [50]:
import holidays

jp_holidays = holidays.JP()


In [51]:
timestamp[11200], timestamp[11200] in jp_holidays


(Timestamp('2022-01-01 16:29:22'), True)

In [52]:
day_of_week = [dt.weekday() for dt in timestamp]
holiday = [1 if dt in jp_holidays else 0 for dt in timestamp]
working_day = [0 if (is_holiday or weekday in [5, 6]) else 1 for (weekday, is_holiday) in zip(day_of_week, holiday)]
week_of_month = [(dt.day - 1) // 7 + 1 for dt in timestamp]

time_features = pd.DataFrame(
    {
        "day_of_week": day_of_week,
        "working_day": working_day,
        "week_of_month": week_of_month,
        "holiday": holiday,
    }
)
time_features


,day_of_week,working_day,week_of_month,holiday
0,0,1,1,0
1,0,1,1,0
2,0,1,1,0
3,0,1,1,0
4,1,1,1,0
...,...,...,...,...
87835,6,0,3,0
87836,6,0,3,0
87837,6,0,3,0
87838,6,0,3,0


In [53]:
full_features = pd.concat([full_features, time_features], axis=1)
full_features = full_features.drop(['timestamp_round1h', 'timestamp_round30m'], axis=1)
full_features.to_csv("data/timezone9/full_features.csv", index=False)
full_features


,device_id,timestamp,lat_long,value,pv_output,solar_radiation,temperature,Precip,humidity,day_of_week,working_day,week_of_month,holiday
0,customer_1,2021-10-04 20:09:38,34.7037561598_137.3584203017,2858.0,NaN,NaN,NaN,NaN,NaN,0,1,1,0
1,customer_1,2021-10-04 22:39:38,34.7037561598_137.3584203017,2958.0,NaN,NaN,NaN,NaN,NaN,0,1,1,0
2,customer_1,2021-10-04 22:49:38,34.7037561598_137.3584203017,2683.0,NaN,NaN,NaN,NaN,NaN,0,1,1,0
3,customer_1,2021-10-04 22:59:38,34.7037561598_137.3584203017,2374.0,NaN,NaN,NaN,NaN,NaN,0,1,1,0
4,customer_1,2021-10-05 03:39:38,34.7037561598_137.3584203017,1808.0,NaN,NaN,NaN,NaN,NaN,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87835,poweru,2022-04-17 23:18:50,34.7037561598_137.3584203017,194.3,0.0,0.0,13.56,0.038462,84.923077,6,0,3,0
87836,poweru,2022-04-17 23:28:50,34.7037561598_137.3584203017,181.8,0.0,0.0,13.56,0.038462,84.923077,6,0,3,0
87837,poweru,2022-04-17 23:38:50,34.7037561598_137.3584203017,209.7,0.0,0.0,13.56,0.038462,84.923077,6,0,3,0
87838,poweru,2022-04-17 23:48:50,34.7037561598_137.3584203017,209.7,0.0,0.0,13.56,0.038462,84.923077,6,0,3,0
